# Markov Model constructor

In this notebook, I want to build a constructor for a Markov model. 

In [13]:
# Put all the imports at the beginning

import numpy as np

Although it's not the only way of defining a Markov model, for the moment, I'm going to do the definition by taking arguments in the constructor that 

In [52]:
class MarkovModel:
    
    def __init__(self, initialStates_ls, transitions_ls, extraStates_ls=[]):
        '''
        Take a list of initial states, and a list of pairs of transitions
        between states. Create a Markov model based on the distribution of
        initial states, and distribution of transitions.
        
        extraStates_ls is a list of additional states which do not appear
        in the two lists initialStates_ls and transitions_ls.
        '''
        
        # First, build the list of all states in the model
        self.states_ls=list({x for x in initialStates_ls}. \
                              union({x for (x, y) in transitions_ls}). \
                              union({y for (x, y) in transitions_ls}). \
                              union(set(extraStates_ls)))
        self.states_ls.sort() # just for aesthetics

        # Now build an array that contains the initial states
        self.initialState_ar=np.array([initialStates_ls.count(state) 
                                       for state in self.states_ls])
        # and normalise the values so the prob.s sum to 1
        self.initialState_ar=self.initialState_ar/np.sum(self.initialState_ar)
        
        # Now build an array that encodes the transitions
        self.transitionMatrix_ar=np.zeros((len(self.states_ls), 
                                           len(self.states_ls)), 
                                          dtype=np.float)  # Normally int, but we're
                                                           # going to normalise
        for (x, y) in transitions_ls:
            self.transitionMatrix_ar[self.states_ls.index(x)][self.states_ls.index(y)]+=1
        for (i, r) in enumerate(self.transitionMatrix_ar):
            if np.sum(self.transitionMatrix_ar[i])>0:
                self.transitionMatrix_ar[i]=self.transitionMatrix_ar[i]/sum(self.transitionMatrix_ar[i])


## Create input of a set of transitions and initial states

We'll assume that the model is to be built from a set of pairs, where each pair (X, Y) represents a single occurrence of a transition from state X to state Y.

In [38]:
transitions_ls=[('a', 'a'), ('a', 'b'), ('a', 'b'), ('a', 'b'), ('a', 'b'), 
                ('a', 'c'), ('a', 'c'), ('a', 'c'), ('a', 'c'), ('a', 'c'), 
                ('b', 'a'), ('b', 'b'), ('b', 'b'), ('b', 'b'), ('b', 'b'), 
                ('b', 'b'), ('b', 'b'), ('b', 'b'), ('b', 'b'), ('b', 'c'),
                ('c', 'a'), ('c', 'a'), ('c', 'a'), ('c', 'b'), ('c', 'b'),
                ('c', 'b'), ('c', 'b'), ('c', 'c'), ('c', 'c'), ('c', 'c')]

In [66]:
transitions_ls=[('a', 'a'), ('a', 'a'), ('a', 'a'), ('a', 'a'), ('a', 'b'), 
                ('a', 'b'), ('a', 'c'), ('a', 'c'), ('a', 'c'), ('a', 'c'), 
                ('b', 'a'), ('b', 'a'), ('b', 'b'), ('b', 'b'), ('b', 'b'), 
                ('b', 'b'), ('b', 'c'), ('b', 'c'), ('b', 'c'), ('b', 'c'),
                ('c', 'a'), ('c', 'a'), ('c', 'a'), ('c', 'b'), ('c', 'b'),
                ('c', 'b'), ('c', 'b'), ('c', 'c'), ('c', 'c'), ('c', 'c')]

And let's shuffle them, just for good measure:

In [67]:
random.shuffle(transitions_ls)
transitions_ls[:5]

[('b', 'c'), ('b', 'a'), ('a', 'a'), ('b', 'b'), ('b', 'b')]

We also need the initial states, which we can also represent as a list.

In [14]:
a=np.array([3,6,5,7,2,1,6])
a

array([3, 6, 5, 7, 2, 1, 6])

In [18]:
a/np.sum(a)

array([ 0.1       ,  0.2       ,  0.16666667,  0.23333333,  0.06666667,
        0.03333333,  0.2       ])

In [53]:
mm=MarkovModel(['a'], transitions_ls, ['b', 'c', 'd', 'e'])

mm.states_ls

['a', 'b', 'c', 'd', 'e']

In [54]:
mm.initialState_ar

array([ 1.,  0.,  0.,  0.,  0.])

In [55]:
mm.transitionMatrix_ar

array([[ 0.1,  0.4,  0.5,  0. ,  0. ],
       [ 0.1,  0.8,  0.1,  0. ,  0. ],
       [ 0.3,  0.4,  0.3,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ]])

## Given the transitions, convert them to a Markov model

The Markov model will consist of an initial distribution of states, and a transition matrix. These will be based on the initial state list and transition lists given in the previous section.

Now get the set of states, and convert them to a list so they can be used for indexing. (Only sorting the list for aesthetic purposes):

In [69]:
states_set={x for (x, y) in transitions_ls}.union({y for (x, y) in transitions_ls}).union({z for z in initialStates_ls})

markovModelStates_ls=sorted(list(states_set))
markovModelStates_ls

['a', 'b', 'c']

Now we create an numpy array, which we will use as the transition matrix:

In [70]:
transitionCount_ar=np.zeros((len(markovModelStates_ls), len(markovModelStates_ls)), dtype=np.int)
transitionCount_ar

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

And populate the count array:

In [71]:
for (x, y) in transitions_ls:
    transitionCount_ar[markovModelStates_ls.index(x)][markovModelStates_ls.index(y)]+=1
    
transitionCount_ar


array([[4, 2, 4],
       [2, 4, 4],
       [3, 4, 3]])

Now create a new array of floats, and populate it with normalised values. To normalise a row, it'll be easiest to define a function

In [72]:
def normalise_1d_array(arrIn_ar):
    # Could check that arrIn_ar.ndim==1
    return(np.array([i/arrIn_ar.sum() for i in arrIn_ar],
                    dtype=np.float))
    
    


Then we can convert the transition counts into a normalised array:

In [73]:
markovModel_ar=np.empty(transitionCount_ar.shape, dtype=np.float)

for i in range(markovModel_ar.shape[0]):
    markovModel_ar[i]=normalise_1d_array(transitionCount_ar[i])

markovModel_ar

array([[ 0.4,  0.2,  0.4],
       [ 0.2,  0.4,  0.4],
       [ 0.3,  0.4,  0.3]])

Now, set up an array of initial states.

In [74]:
initialStateCounts_ls=[initialStates_ls.count(state) for state in markovModelStates_ls]

initialStateCounts_ls

[1, 0, 0]

In [75]:
ar=normalise_1d_array(np.array(initialStateCounts_ls))
ar

array([ 1.,  0.,  0.])

And now, we can take the logs of the values so we can avoid vanishing probabilities.

In [76]:
initialStates_ar=np.log(ar)
initialStates_ar

/Users/alistairwillis/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


array([  0., -inf, -inf])

In [77]:
transitionMatrix_ar=np.log(markovModel_ar)
transitionMatrix_ar

array([[-0.91629073, -1.60943791, -0.91629073],
       [-1.60943791, -0.91629073, -0.91629073],
       [-1.2039728 , -0.91629073, -1.2039728 ]])

## Now calculate the state after a Markov step

What we want is a function that takes a transition matrix, and a probabilistic distribution of states, and returns a distribution of states in the next step, and an array of the previous states.

Let's assume that we're working fully with logged versions at the moment.

In [78]:
def next_step(stateIn_ar, transitionIn_ar):
    '''
    Takes an input state and a transition matrix, and returns
    an output state.
    
    Both stateIn_ar and transitionIn_ar are expressed as logs.
    '''
    
    stateOut_ar=np.empty(stateIn_ar.shape)
    transOut_ar=np.zeros(stateIn_ar.shape, dtype=np.int)
    
    for (i, x) in enumerate(stateIn_ar):
        calculateTransitions_ar=stateIn_ar + transitionMatrix_ar[i]
        argmax_i=np.argmax(calculateTransitions_ar)
        
        stateOut_ar[i]=calculateTransitions_ar[argmax_i]
        transOut_ar[i]=argmax_i
        
        
    return (stateOut_ar, transOut_ar)
    

In [79]:
next_step(initialStates_ar, transitionMatrix_ar)

(array([-0.91629073, -1.60943791, -1.2039728 ]), array([0, 0, 0]))

In [80]:
u=next_step(initialStates_ar, transitionMatrix_ar)[0]
next_step(u, transitionMatrix_ar)

(array([-1.83258146, -2.12026354, -2.12026354]), array([0, 2, 0]))

In [82]:
def most_likely_paths(stateIn_ar, transitionIn_ar, steps=10):
    '''
    Calculate the most likely path to each state from stateIn_ar
    after steps transitions
    '''
    paths=[]
    currentState_ar=stateIn_ar
    for i in range(steps):
        (currentState_ar, trans_ar)=next_step(currentState_ar, transitionIn_ar)
        paths.append(list(trans_ar))
        
    return paths
        

In [83]:
most_likely_paths(initialStates_ar, transitionMatrix_ar, 10)

[[0, 0, 0],
 [0, 2, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 2, 0],
 [0, 1, 0],
 [0, 1, 1],
 [0, 1, 1]]

In [84]:
u=_

In [85]:
np.array(u).T

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 1, 1, 1, 1, 2, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])